In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import torch

# Check if CUDA is available
print("CUDA available:", torch.cuda.is_available())

# Check the CUDA version used by PyTorch
print("PyTorch CUDA version:", torch.version.cuda)

# Check the installed GPU
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("No GPU detected")


CUDA available: True
PyTorch CUDA version: 12.6
GPU: NVIDIA GeForce RTX 4070


In [ ]:
def load_images_from_directory(directory, target_size=(1024, 1024)):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if os.path.isdir(label_dir):
            for img_name in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_name)
                img = image.load_img(img_path, target_size=target_size)
                img_array = image.img_to_array(img)
                images.append(img_array)
                labels.append(label)
    return images, labels

train_images, train_labels = load_images_from_directory(r"C:\Users\antoi\Documents\Nell_Antoine_Project\DATA")
validation_images, validation_labels = load_images_from_directory(r"C:\Users\antoi\Documents\Nell_Antoine_Project\DATA")